<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/21B_Tarea_Introduccion_a_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 21. Tarea Intro a Spark


In [2]:
from pyspark.sql import SparkSession
import os

os.makedirs("/content/spark-tmp", exist_ok=True)

spark = (
    SparkSession.builder
    .appName("colab")
    .master("local[*]")
    .config("spark.ui.enabled", "false")      # UI not needed in Colab
    .config("spark.driver.memory", "2g")      # avoid OOM-killed JVM
    .config("spark.local.dir", "/content/spark-tmp")
    .getOrCreate()
)

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [1]:
df = spark.read.csv('covidmex.csv',header=True)

# Nota como, si no le doy un .show(), spark solo "prepara" la lectura de la tabla. Esa es la famosa Lazy Evaluation

NameError: name 'spark' is not defined

### ¿Cuál es el problema de una implementación de este tipo?

In [ ]:
from pyspark.sql import functions as F

# 1. Agregar la columna "PAIS" con el valor "Mexico"
df = df.withColumn("PAIS", F.lit("Mexico"))

# 2. Eliminar la columna "lengthofday"
df = df.drop("lengthofday")

# 3. Crear un nuevo DataFrame 'ubicacion' con las columnas deseadas
ubicacion = df.select("lat", "long", "ENTIDAD_RES", "PAIS")

# 4. Filtrar 'ubicacion' para quedarte solo con los valores de ENTIDAD_RES == 9
cdmx = ubicacion.filter(ubicacion.ENTIDAD_RES == 9)

# 5. Seleccionar los primeros 5 valores y convertirlos a un DataFrame de Pandas
df_pandas = cdmx.limit(5).toPandas()

# Mostrar el DataFrame de Pandas
print(df_pandas)

                 lat                long ENTIDAD_RES    PAIS
0  19.20155345588235  -99.20180252450989           9  Mexico
1          19.482945          -99.113471           9  Mexico
2          19.359004          -99.092623           9  Mexico
3          19.395901          -99.097613           9  Mexico
4  19.20155345588235  -99.20180252450989           9  Mexico


Nota cómo creamos varias tablas intermedias - mala idea en Spark, sobretodo por el volumen de la información que se suele manejar. Además, "rompemos" la optimización de Spark.

¿Qué sería mejor?

In [ ]:
ubicacion = df.withColumn("PAIS", F.lit("Mexico"))\         # 1. Agregar la columna "PAIS" con el valor "Mexico"
              .drop("lengthofday")\                         # 2. Eliminar la columna "lengthofday"
              .select("lat", "long", "ENTIDAD_RES", "PAIS") # 3. Crear un nuevo DataFrame 'ubicacion' con las columnas deseadas

# 4. Filtrar 'ubicacion' para quedarte solo con los valores de ENTIDAD_RES == 9
cdmx = ubicacion.filter(ubicacion.ENTIDAD_RES == 9)

# 5. Seleccionar los primeros 5 valores y convertirlos a un DataFrame de Pandas
df_pandas = cdmx.limit(5).toPandas()

# Mostrar el DataFrame de Pandas
print(df_pandas)

O mejor aún

In [ ]:
df_pandas = df.withColumn("PAIS", F.lit("Mexico"))\           # 1. Agregar la columna "PAIS" con el valor "Mexico"
              .drop("lengthofday")\                           # 2. Eliminar la columna "lengthofday"
              .select("lat", "long", "ENTIDAD_RES", "PAIS")\  # 3. Crear un nuevo DataFrame 'ubicacion' con las columnas deseadas
              .filter(ubicacion.ENTIDAD_RES == 9)             # 4. Filtrar 'ubicacion' para quedarte solo con los valores de ENTIDAD_RES == 9
              .limit(5).toPandas()                            # 5. Seleccionar los primeros 5 valores y convertirlos a un DataFrame de Pandas

print(df_pandas)